# Task 1a: Perform Text Generation

In this notebook, you learn how to use Large Language Model (LLM) to generate an email response to a customer who provided negative feedback on the quality of customer service they received from the support engineer. In this notebook, you generate an email with a thank you note based on the customer's previous email. You use the Amazon Nova Lite model using the Amazon Bedrock API with Boto3 client.

The prompt used in this task is called a zero-shot prompt. In a zero-shot prompt, you describe the task or desired output to the language model in plain language. The model then uses its pre-trained knowledge and capabilities to generate a response or complete the task based solely on the provided prompt.

#### Scenario
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers apologizing for the poor service and to regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

## Task 1a.1: Environment setup

In this task, you set up your environment.

In [1]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Task 1a.2: Generate text

In this task, you prepare an input for the Amazon Bedrock service to generate an email.

In [2]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [3]:
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 8192,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}) 

Next, you use the Amazon Nova Lite model.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** Amazon Nova Lite supports a context window of ~300k tokens and accepts the following parameters:
- `messages`: Array of message objects with role and content
- `inferenceConfig`: These are the parameters that model will take into account while generating the output.

The Amazon Bedrock API provides you with an API `invoke_model` which accepts the following:
- `modelId`: This is the model ARN for the various foundation models available under Amazon Bedrock
- `accept`: The type of input request
- `contentType`: The content type of the output
- `body`: A json string consisting of the prompt and the configurations

Refer to [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids.

## Task 1a.3: Invoke the Amazon Nova Lite language model

In this task, you explore how the model generates an output based on the prompt created earlier.

### Complete Output Generation

This email is generated using the Amazon Nova Lite model by understanding the input request and utilizing its inherent understanding of different modalities. The request to the API is synchronous and waits for the entire output to be generated by the model.

In [4]:
#invoke model
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [5]:
# Print the complete response from Nova Lite
print(outputText)


Subject: Addressing Your Recent Experience with Our Customer Support

Dear John Doe,

I hope this email finds you well.

I am writing to you personally regarding the recent feedback you provided about your experience with our customer support team. First and foremost, I want to express my sincere apologies for any inconvenience or frustration you may have encountered. Your satisfaction is of utmost importance to us, and it is clear that we fell short of your expectations.

We take all feedback seriously, and I want to assure you that we are committed to making things right. I have reviewed the details of your case and would like to discuss the issues you faced in more detail. If possible, could you please provide me with a bit more information about the specific problems you encountered? This will help us to understand the situation better and take appropriate action.

Our goal is to ensure that all our customers have a positive experience with AnyCompany, and we are dedicated to impro

### Streaming Output Generation

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. This email is generated by invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

In [6]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if 'contentBlockDelta' in chunk_obj:
                    text = chunk_obj['contentBlockDelta']['delta']['text']
                    output.append(text)
                    print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                    i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**
Subject

		**Chunk 2**
: Addressing

		**Chunk 3**
 Your

		**Chunk 4**
 Recent

		**Chunk 5**
 Feedback

		**Chunk 6**




		**Chunk 7**
Dear John

		**Chunk 8**
 Doe

		**Chunk 9**
,

I hope this

		**Chunk 10**
 message finds

		**Chunk 11**
 you well

		**Chunk 12**
.



		**Chunk 13**
I

		**Chunk 14**
 am writing to you in

		**Chunk 15**
 response to the

		**Chunk 16**
 feedback

		**Chunk 17**
 you recently

		**Chunk 18**
 provided regarding

		**Chunk 19**
 the

		**Chunk 20**
 service you

		**Chunk 21**
 received

		**Chunk 22**
 from our customer support

		**Chunk 23**
 engineer

		**Chunk 24**
.

		**Chunk 25**
 First

		**Chunk 26**
 and foremost

		**Chunk 27**
, I want

		**Chunk 28**
 to sincerely apologize

		**Chunk 29**
 for any

		**Chunk 30**
 inconvenience or

		**Chunk 31**
 frustration you experienced

		**Chunk 32**
 during your interaction

		**Chunk 33**
 with our team

		**Chunk 34**
. 



		**Chunk 35**
At Any

		**Chunk 36**
Company, we s

The stream with response approach helps to quickly obtain the output of the model and allows the service to complete it as you read. This assists in use cases where you request the model to generate longer pieces of text. You can later combine all the chunks generated to form the complete output and use it for your use case. 

In [7]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**

Subject: Addressing Your Recent Feedback

Dear John Doe,

I hope this message finds you well.

I am writing to you in response to the feedback you recently provided regarding the service you received from our customer support engineer. First and foremost, I want to sincerely apologize for any inconvenience or frustration you experienced during your interaction with our team. 

At AnyCompany, we strive to provide exceptional customer service, and it is clear that we fell short of your expectations on this occasion. Your feedback is incredibly valuable to us, as it helps us identify areas where we can improve and ensure that we are meeting the high standards our customers expect and deserve.

I have reviewed the details of your case and would like to personally address the issues you encountered. To this end, I have taken the following steps:

1. **Investigation**: We have thoroughly investigated the specific issues you mentioned. Our team is committed to understan


You have now experimented with using the boto3 SDK, which provides basic exposure to the Amazon Bedrock API. Using this API, you have seen the use case of generating an email to respond to a customer's negative feedback.

### Try it yourself
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task1b.ipynb**.